# Marking system for second coursework


In [2]:
import numpy as np
# this is the only import you are allowed in your code

In [3]:
class ABSGreedyRuleInductionModel:
    
    def __init__(self, maxRules=5, increments=100):
        # you probably won't need to change this
        # empty array to hold model (ruleset)
        self._maxRules= maxRules
        self._model = np.empty((self._maxRules,4))
        self._numRules=0
        # number of threshold values to consider for each feature
        self._increments=increments
        #empty model makes no correct or incorrect predictions
        self.score=0
    
    def GetModel(self):
        return self._model[:self._numRules,: ]
        
    def fit(self, train_X,train_y):
        # your code here
        
        # when you finish your set of rules should be stored in self._model
        # and the number of rules in self._numRules
        pass
    
    def predict(self,examples):
        ypred = np.zeros(examples.shape[0])
        # your code here
        return ypred       
    
    def preprocess(self, data):
        # your code goes here
        #to calculate the set of threshold values to loop over for each feature
        pass
    
    

In [104]:
class GreedyRuleInductionModel:
    
    def __init__(self,maxRules=5, increments=10):
 
        #read some user-configurablew paramerers
        ## maximum size of ruleset that is our model
        self.maxRules= maxRules
        ## number of threshold values to consider for each feature
        self.numThresholds=increments
        
        # initialise currentModel to be an empty array with no rules that scores 0
        self.model = np.empty((self.maxRules,4),dtype=np.uint)
        self.numRules=0
        self.score=0

    
    def GetModel(self):
        return self.model[:self.numRules,: ]
        
    def fit( self,train_X,train_y):
         # store the set of different labels - don;t assume theyare 0,12,2 etc
        self.labels = np.unique(train_y)

        # preprocess the data to compute threasholds to be used in rules
        self.preprocess(train_X)
        numTrItems=train_X.shape[0]

        #WHILE (currentModel.score<trainingsetSize) DO 
        while (self.score<numTrItems and self.numRules<self.maxRules):
            print("Adding to model with {} rules which score {}".format(self.numRules,self.score))
            # SET bestchild = emptyModel
            bestChild= self.model.copy()
            bestChildScore=self.score
            
            #FOR newRule in  (all_possible_rules)
            for feature in range(self.numFeatures):
                for operator in range (3):
                    for threshold in range (self.numThresholds):
                        for label in range(len(self.labels)):
                            newRule= np.array ( [feature,operator,threshold,label])
                            #   SET newModel = COPY(currentModel)
                            newModel = self.model.copy()
                            ##  SET newModel = ADDRULE (newModel, newRule)
                            newModel[self.numRules]= newRule
                            #  SET score = SCORE(newModel)
                            newScore = self.Score(newModel, self.numRules+1,train_X,train_y)
                            #  IF (newModel.score > bestChild.score)
                            if (newScore>bestChildScore):
                                #SET bestChild= COPY(newModel)
                                bestChild = newModel.copy()
                                bestChildScore = newScore
                                #print("new best rule for this iteration classifies {}".format(bestChildScore))

            #IF (bestChild.score > currentModel.score)
            if (bestChildScore > self.score):
                #SET currentModel=COPY (bestChild)
                self.model = bestChild.copy()
                self.numRules += 1
                self.score = bestChildScore
            else:
                print("exiting training loop becuase no improving rule could be found")
                break

                
        print("Model has {} rules and a training set accuracy of {}".format(self.numRules,(100*self.score/numTrItems)))


    
    def predict(self, examples):
        ypred = np.zeros(examples.shape[0],dtype=np.uint)
        for item in range (examples.shape[0]):
            prediction = -1 # i'm going ot use -1 to denote "NO_PREDICTION"
            for currentRule in range (self.numRules):
                if ( self.ItemMatchesRule(self.model[currentRule], examples[item] )==True):
                    prediction = self.model[currentRule][3]
                    break
            ypred[item]= self.labels[prediction]
        return ypred
    
    
    def preprocess(self, data):
        # your code goes here
        #to calculate the set of threshold values to loop over for each feature
        numItems = data.shape[0]
        self.numFeatures= data.shape[1]
        self.thresholds = np.empty((self.numFeatures, self.numThresholds))
        maxValues = np.max(data,axis=0)
        minValues = np.min( data, axis=0)
        for feat in range (self.numFeatures):
            thisFeatureIncrement = (maxValues[feat] - minValues[feat] )/self.numThresholds
            for thresh in range ( self.numThresholds):
                self.thresholds[feat][thresh] = minValues[feat] + thresh * thisFeatureIncrement

        
    def ItemMatchesRule( self,rule, item):

        if len(rule) != 4:
            print("error in ItemMatchesRule, rule must contain exactly four values")
            return False
        feature = int(rule[0])
        if( feature <0 or feature> self.numFeatures):
            print('invalid feature id encountered')
            return False
        operator = int(rule[1])
        if( operator <0 or operator>2):
            print('invalid operator id encountered')
            return False
        thresholdId = int(rule[2])
        if( thresholdId <0 or thresholdId>= self.numThresholds):
            print('invalid threshold index encountered')
            return False
 

        threshold = self.thresholds[feature][thresholdId]
        
        #print('rule is feature {} operator {} thrshold {}'.format(feature,operator,threshold))
        #print('item is {}'.format(item))
        
        if( (operator==0) and  (item [feature] < threshold) ):
            return(True)
        elif (operator ==1 and  ( item [feature] == threshold) ):
            return True
        elif ( (operator ==2) and ( item [feature]> threshold)):
            return True
        else:
            return False
        
    def Score( self,model, numRules, dataset_X,dataset_y):
        numToPredict = dataset_X.shape[0]
        numCorrect=0
        for item in range( numToPredict):
            prediction = -1 # i'm going ot use -1 to denote "NO_PREDICTION"
            for currentRule in range (numRules):
                if ( self.ItemMatchesRule(model[currentRule], dataset_X[item] )==True):
                    prediction = self.labels[model[currentRule][3]]
                    break
            if (prediction>=0 and  prediction != dataset_y[item] ):
                numCorrect = -1
                break
            elif (prediction == dataset_y[item]):
                numCorrect +=1
        return numCorrect
            
    
 
    

In [105]:
def TestAlgorithmOnDataset(X_train, y_train,X_test,y_test):
    
    myModel = GreedyRuleInductionModel(maxRules=5)

    myModel.fit(X_train,y_train)
    #print("learned model is {}".format(myModel.GetModel()))

    ypred = myModel.predict(X_test)

    confusionMatrix = np.zeros((4,4),np.uint)
    accuracy = 0.0
    for i in range(len(y_test)):
        actual = int(y_test[i])
        predicted = int(ypred[i])
        confusionMatrix[actual][predicted] += 1
        if(actual==predicted):
            accuracy += 1.0
    accuracy = accuracy*100/len(y_test)
    print(" accuracy on this dataset is {}%".format(accuracy))
    print(confusionMatrix)

In [103]:
# simple training on the iris data

# make train/test split 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris


irisX,irisy = load_iris(return_X_y = True)
irisy = irisy 

for repetition in range (5):
    X_train, X_test, y_train, y_test = train_test_split(irisX, irisy, test_size=0.33,stratify=irisy)

    TestAlgorithmOnDataset(X_train,y_train,X_test,y_test)

Adding to model with 0 rules which score 0
Adding to model with 1 rules which score 33
Adding to model with 2 rules which score 52
Adding to model with 3 rules which score 70
Adding to model with 4 rules which score 73
 accuracy on this dataset is 84.0%
[[17  0  0  0]
 [ 0  8  8  0]
 [ 0  0 17  0]
 [ 0  0  0  0]]
Adding to model with 0 rules which score 0
Adding to model with 1 rules which score 33
Adding to model with 2 rules which score 56
Adding to model with 3 rules which score 66
Adding to model with 4 rules which score 74
 accuracy on this dataset is 86.0%
[[17  0  0  0]
 [ 0 11  6  0]
 [ 0  1 15  0]
 [ 0  0  0  0]]
Adding to model with 0 rules which score 0
Adding to model with 1 rules which score 33
Adding to model with 2 rules which score 52
Adding to model with 3 rules which score 69
Adding to model with 4 rules which score 73
 accuracy on this dataset is 84.0%
[[17  0  0  0]
 [ 0  8  8  0]
 [ 0  0 17  0]
 [ 0  0  0  0]]
Adding to model with 0 rules which score 0
Adding to mo